# MINI-PROJ-01

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(32)
tf.random.set_seed(32)

In [2]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10891836372139938127
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4857331712
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14905290540825763041
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [4]:
X_train = X_train_full / 255.
y_train = y_train_full.astype(float)

In [5]:
tf.__version__

'2.9.1'

In [6]:
X_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [7]:
y_train

array([9., 0., 0., ..., 3., 0., 5.])

In [8]:
print(X_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [9]:
print(X_test.shape)

(10000, 28, 28)


In [11]:
def build_model():
    from functools import partial

    DefaultConv2D = partial(keras.layers.Conv2D,
                            kernel_size=2, kernel_initializer = 'lecun_normal',
                            activation='relu', padding="SAME")

    model = keras.models.Sequential([
        DefaultConv2D(filters=32, kernel_size=3, input_shape=[28, 28,1]),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        
        keras.layers.Flatten(),
        keras.layers.Dense(units=128, activation='selu'),
        keras.layers.BatchNormalization(),
#         keras.layers.Dropout(0.5),
        keras.layers.Dense(units=64, activation='selu'),
        keras.layers.BatchNormalization(),
#         keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
    ])
    
    
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model

In [12]:
k = 10
num_val = len(X_train) // k
num_epochs = 50
all_scores = []


for i in range(k):
    print('processing fold #', i)

    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    with tf.device("/device:GPU:0"):
        model.fit(X_train_part, y_train_part,
                  epochs=num_epochs, verbose=1)
    history = model.fit(X_train_part, y_train_part,
                  epochs=num_epochs, verbose=1)
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
Epoch 1/50
1688/1688 [==============================] - 34s 16ms/step - loss: 0.3730 - accuracy: 0.8642
Epoch 2/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.2533 - accuracy: 0.9076
Epoch 3/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.2177 - accuracy: 0.9203
Epoch 4/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.1892 - accuracy: 0.9314
Epoch 5/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.1599 - accuracy: 0.9420
Epoch 6/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.1378 - accuracy: 0.9498
Epoch 7/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.1176 - accuracy: 0.9573
Epoch 8/50
1688/1688 [==============================] - 26s 15ms/step - loss: 0.0998 - accuracy: 0.9644
Epoch 9/50
1688/1688 [==============================] - 26s 15ms/step - loss: 0.0842 - accuracy: 0.9692
Epoch 10/50
1688/1688 [=====================

1688/1688 [==============================] - 27s 16ms/step - loss: 0.0053 - accuracy: 0.9981
Epoch 30/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0072 - accuracy: 0.9976
Epoch 31/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0053 - accuracy: 0.9981
Epoch 32/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0057 - accuracy: 0.9981
Epoch 33/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0064 - accuracy: 0.9979
Epoch 34/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0052 - accuracy: 0.9984
Epoch 35/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0055 - accuracy: 0.9981
Epoch 36/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0052 - accuracy: 0.9982
Epoch 37/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0056 - accuracy: 0.9980
Epoch 38/50
1688/1688 [==============================] - 26s 16ms/s

1688/1688 [==============================] - 26s 16ms/step - loss: 0.0105 - accuracy: 0.9965
Epoch 7/50
1688/1688 [==============================] - 26s 15ms/step - loss: 0.0069 - accuracy: 0.9975
Epoch 8/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0108 - accuracy: 0.9962
Epoch 9/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0069 - accuracy: 0.9980
Epoch 10/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0092 - accuracy: 0.9969
Epoch 11/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0087 - accuracy: 0.9970
Epoch 12/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0073 - accuracy: 0.9974
Epoch 13/50
1688/1688 [==============================] - 26s 15ms/step - loss: 0.0086 - accuracy: 0.9972
Epoch 14/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0068 - accuracy: 0.9978
Epoch 15/50
1688/1688 [==============================] - 27s 16ms/step

1688/1688 [==============================] - 27s 16ms/step - loss: 0.0160 - accuracy: 0.9947
Epoch 34/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0154 - accuracy: 0.9943
Epoch 35/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0162 - accuracy: 0.9943
Epoch 36/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0149 - accuracy: 0.9948
Epoch 37/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0153 - accuracy: 0.9947
Epoch 38/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0140 - accuracy: 0.9952
Epoch 39/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0133 - accuracy: 0.9954
Epoch 40/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0131 - accuracy: 0.9956
Epoch 41/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0127 - accuracy: 0.9957
Epoch 42/50
1688/1688 [==============================] - 27s 16ms/s

1688/1688 [==============================] - 27s 16ms/step - loss: 0.0720 - accuracy: 0.9741
Epoch 11/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0638 - accuracy: 0.9761
Epoch 12/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0575 - accuracy: 0.9783
Epoch 13/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0494 - accuracy: 0.9822
Epoch 14/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0451 - accuracy: 0.9833
Epoch 15/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0409 - accuracy: 0.9848
Epoch 16/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0362 - accuracy: 0.9873
Epoch 17/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0363 - accuracy: 0.9870
Epoch 18/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0324 - accuracy: 0.9881
Epoch 19/50
1688/1688 [==============================] - 27s 16ms/s

...

1688/1688 [==============================] - 26s 16ms/step - loss: 0.0311 - accuracy: 0.9888
Epoch 20/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0264 - accuracy: 0.9902
Epoch 21/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0269 - accuracy: 0.9902
Epoch 22/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0273 - accuracy: 0.9906
Epoch 23/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0218 - accuracy: 0.9924
Epoch 24/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0230 - accuracy: 0.9916
Epoch 25/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0218 - accuracy: 0.9923
Epoch 26/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0194 - accuracy: 0.9932
Epoch 27/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0209 - accuracy: 0.9930
Epoch 28/50
1688/1688 [==============================] - 26s 15ms/s

1688/1688 [==============================] - 26s 16ms/step - loss: 0.0042 - accuracy: 0.9986
Epoch 48/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0046 - accuracy: 0.9984
Epoch 49/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0058 - accuracy: 0.9980
Epoch 50/50
188/188 [==============================] - 1s 6ms/step - loss: 0.4880 - accuracy: 0.9330
processing fold # 7
Epoch 1/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.3679 - accuracy: 0.8686
Epoch 2/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.2546 - accuracy: 0.9070
Epoch 3/50
1688/1688 [==============================] - 26s 15ms/step - loss: 0.2151 - accuracy: 0.9216
Epoch 4/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.1864 - accuracy: 0.9314
Epoch 5/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.1604 - accuracy: 0.9414
Epoch 6/50
1688/1688 [==============================] - 

1688/1688 [==============================] - 26s 16ms/step - loss: 0.0091 - accuracy: 0.9974
Epoch 25/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0063 - accuracy: 0.9981
Epoch 26/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0053 - accuracy: 0.9986
Epoch 27/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0061 - accuracy: 0.9979
Epoch 28/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0077 - accuracy: 0.9974
Epoch 29/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0053 - accuracy: 0.9981
Epoch 30/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0069 - accuracy: 0.9976
Epoch 31/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0064 - accuracy: 0.9979
Epoch 32/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0059 - accuracy: 0.9981
Epoch 33/50
1688/1688 [==============================] - 26s 16ms/s

1688/1688 [==============================] - 27s 16ms/step - loss: 0.0103 - accuracy: 0.9966
Epoch 2/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0121 - accuracy: 0.9962
Epoch 3/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0091 - accuracy: 0.9972
Epoch 4/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0101 - accuracy: 0.9967
Epoch 5/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0112 - accuracy: 0.9958
Epoch 6/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0068 - accuracy: 0.9974
Epoch 7/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0106 - accuracy: 0.9964
Epoch 8/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0072 - accuracy: 0.9977
Epoch 9/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0083 - accuracy: 0.9974
Epoch 10/50
1688/1688 [==============================] - 27s 16ms/step - lo

1688/1688 [==============================] - 27s 16ms/step - loss: 0.0199 - accuracy: 0.9931
Epoch 29/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0168 - accuracy: 0.9941
Epoch 30/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0179 - accuracy: 0.9939
Epoch 31/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0165 - accuracy: 0.9944
Epoch 32/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0168 - accuracy: 0.9939
Epoch 33/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0159 - accuracy: 0.9942
Epoch 34/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0167 - accuracy: 0.9943
Epoch 35/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0141 - accuracy: 0.9953
Epoch 36/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0138 - accuracy: 0.9956
Epoch 37/50
1688/1688 [==============================] - 27s 16ms/s

In [13]:
all_scores

[0.9325000047683716,
 0.9313333630561829,
 0.9284999966621399,
 0.9330000281333923,
 0.9350000023841858,
 0.9356666803359985,
 0.9330000281333923,
 0.9381666779518127,
 0.9259999990463257,
 0.9358333349227905]

In [14]:
np.mean(all_scores)

0.9329000115394592

In [15]:
def build_model():
    from functools import partial

    DefaultConv2D = partial(keras.layers.Conv2D,
                            kernel_size=2, kernel_initializer = 'lecun_normal',
                            activation='selu', padding="SAME")

    model = keras.models.Sequential([
        DefaultConv2D(filters=32, kernel_size=3, input_shape=[28, 28,1]),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        
        keras.layers.Flatten(),
        keras.layers.Dense(units=128, activation='selu'),
        keras.layers.BatchNormalization(),
#         keras.layers.Dropout(0.5),
        keras.layers.Dense(units=64, activation='selu'),
        keras.layers.BatchNormalization(),
#         keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
    ])
    
    
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model

In [16]:
k = 10
num_val = len(X_train) // k
num_epochs = 50
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=1, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
Epoch 1/50
1688/1688 [==============================] - 28s 16ms/step - loss: 0.3864 - accuracy: 0.8612
Epoch 2/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.2668 - accuracy: 0.9018
Epoch 3/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.2291 - accuracy: 0.9160
Epoch 4/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.2015 - accuracy: 0.9266
Epoch 5/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.1727 - accuracy: 0.9371
Epoch 6/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.1483 - accuracy: 0.9447
Epoch 7/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.1288 - accuracy: 0.9525
Epoch 8/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.1099 - accuracy: 0.9600
Epoch 9/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0937 - accuracy: 0.9650
Epoch 10/50
1688/1688 [=====================

Epoch 28/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0176 - accuracy: 0.9938
Epoch 29/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0187 - accuracy: 0.9936
Epoch 30/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0160 - accuracy: 0.9943
Epoch 31/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0164 - accuracy: 0.9943
Epoch 32/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0149 - accuracy: 0.9946
Epoch 33/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0163 - accuracy: 0.9946
Epoch 34/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0139 - accuracy: 0.9950
Epoch 35/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0122 - accuracy: 0.9958
Epoch 36/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0141 - accuracy: 0.9948
Epoch 37/50
1688/1688 [==============================] 

1688/1688 [==============================] - 27s 16ms/step - loss: 0.2047 - accuracy: 0.9256
Epoch 5/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.1755 - accuracy: 0.9359
Epoch 6/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.1508 - accuracy: 0.9437
Epoch 7/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.1280 - accuracy: 0.9529
Epoch 8/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.1086 - accuracy: 0.9602
Epoch 9/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0933 - accuracy: 0.9654
Epoch 10/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0779 - accuracy: 0.9710
Epoch 11/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0656 - accuracy: 0.9763
Epoch 12/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0601 - accuracy: 0.9784
Epoch 13/50
1688/1688 [==============================] - 27s 16ms/step -

1688/1688 [==============================] - 27s 16ms/step - loss: 0.0144 - accuracy: 0.9950
Epoch 32/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0165 - accuracy: 0.9940
Epoch 33/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0159 - accuracy: 0.9944
Epoch 34/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0155 - accuracy: 0.9945
Epoch 35/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0128 - accuracy: 0.9956
Epoch 36/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0164 - accuracy: 0.9945
Epoch 37/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0132 - accuracy: 0.9957
Epoch 38/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0125 - accuracy: 0.9957
Epoch 39/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0122 - accuracy: 0.9961
Epoch 40/50
1688/1688 [==============================] - 27s 16ms/s

1688/1688 [==============================] - 26s 16ms/step - loss: 0.1253 - accuracy: 0.9547
Epoch 8/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.1063 - accuracy: 0.9610
Epoch 9/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0884 - accuracy: 0.9675
Epoch 10/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0743 - accuracy: 0.9720
Epoch 11/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0654 - accuracy: 0.9758
Epoch 12/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0575 - accuracy: 0.9786
Epoch 13/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0482 - accuracy: 0.9825
Epoch 14/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0443 - accuracy: 0.9840
Epoch 15/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0401 - accuracy: 0.9856
Epoch 16/50
1688/1688 [==============================] - 26s 16ms/ste

1688/1688 [==============================] - 27s 16ms/step - loss: 0.0141 - accuracy: 0.9947
Epoch 35/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0162 - accuracy: 0.9948
Epoch 36/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0125 - accuracy: 0.9959
Epoch 37/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0145 - accuracy: 0.9950
Epoch 38/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0138 - accuracy: 0.9956
Epoch 39/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0115 - accuracy: 0.9960
Epoch 40/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0129 - accuracy: 0.9954
Epoch 41/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0116 - accuracy: 0.9962
Epoch 42/50
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0128 - accuracy: 0.9956
Epoch 43/50
1688/1688 [==============================] - 27s 16ms/s

1688/1688 [==============================] - 27s 16ms/step - loss: 0.0744 - accuracy: 0.9729
Epoch 11/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0648 - accuracy: 0.9764
Epoch 12/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0561 - accuracy: 0.9794
Epoch 13/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0481 - accuracy: 0.9826
Epoch 14/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0431 - accuracy: 0.9837
Epoch 15/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0409 - accuracy: 0.9855
Epoch 16/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0350 - accuracy: 0.9872
Epoch 17/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0335 - accuracy: 0.9882
Epoch 18/50
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0310 - accuracy: 0.9892
Epoch 19/50
1688/1688 [==============================] - 27s 16ms/s

In [17]:
all_scores

[0.9279999732971191,
 0.9336666464805603,
 0.9236666560173035,
 0.9306666851043701,
 0.918666660785675,
 0.9213333129882812,
 0.9300000071525574,
 0.9338333606719971,
 0.9256666898727417,
 0.9291666746139526]

In [18]:
np.mean(all_scores)

0.9274666666984558

In [17]:
def build_model():
    from functools import partial

    DefaultConv2D = partial(keras.layers.Conv2D,
                            kernel_size=2, kernel_initializer = 'he_normal',
                            activation='relu', padding="SAME")

    model = keras.models.Sequential([
        DefaultConv2D(filters=64, kernel_size=3, input_shape=[28, 28,1]),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        
        keras.layers.Flatten(),
        keras.layers.Dense(units=128, activation='selu'),
        keras.layers.BatchNormalization(),
#         keras.layers.Dropout(0.5),
        keras.layers.Dense(units=64, activation='selu'),
        keras.layers.BatchNormalization(),
#         keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
    ])
    
    
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model

In [20]:
k = 10
num_val = len(X_train) // k
num_epochs = 35
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=1, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
Epoch 1/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.3928 - accuracy: 0.8599
Epoch 2/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.2727 - accuracy: 0.9015
Epoch 3/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.2310 - accuracy: 0.9141
Epoch 4/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.2040 - accuracy: 0.9253
Epoch 5/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.1773 - accuracy: 0.9343
Epoch 6/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.1573 - accuracy: 0.9425
Epoch 7/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.1365 - accuracy: 0.9494
Epoch 8/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.1186 - accuracy: 0.9561
Epoch 9/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.1041 - accuracy: 0.9615
Epoch 10/35
1688/1688 [=====================

Epoch 7/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.1363 - accuracy: 0.9504
Epoch 8/35
1688/1688 [==============================] - 33s 19ms/step - loss: 0.1183 - accuracy: 0.9567
Epoch 9/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.1048 - accuracy: 0.9609
Epoch 10/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0910 - accuracy: 0.9664
Epoch 11/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0779 - accuracy: 0.9714
Epoch 12/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0689 - accuracy: 0.9748
Epoch 13/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0597 - accuracy: 0.9781
Epoch 14/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0559 - accuracy: 0.9799
Epoch 15/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0484 - accuracy: 0.9825
Epoch 16/35
1688/1688 [==============================] - 3

1688/1688 [==============================] - 32s 19ms/step - loss: 0.0343 - accuracy: 0.9876
Epoch 20/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0317 - accuracy: 0.9884
Epoch 21/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0299 - accuracy: 0.9891
Epoch 22/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0284 - accuracy: 0.9900
Epoch 23/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0283 - accuracy: 0.9894
Epoch 24/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0256 - accuracy: 0.9913
Epoch 25/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0229 - accuracy: 0.9920
Epoch 26/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0235 - accuracy: 0.9915
Epoch 27/35
1688/1688 [==============================] - 32s 19ms/step - loss: 0.0213 - accuracy: 0.9925
Epoch 28/35
1688/1688 [==============================] - 32s 19ms/s

In [21]:
all_scores

[0.9334999918937683,
 0.9244999885559082,
 0.925000011920929,
 0.9276666641235352,
 0.9350000023841858,
 0.9326666593551636,
 0.9358333349227905,
 0.9348333477973938,
 0.9316666722297668,
 0.934333324432373]

In [22]:
np.mean(all_scores)

0.9314999997615814

In [20]:
k = 10
num_val = len(X_train) // k
num_epochs = 40
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 1ms/step - loss: 0.3041 - accuracy: 0.8907
processing fold # 1
188/188 [==============================] - 0s 1ms/step - loss: 0.3015 - accuracy: 0.8913
processing fold # 2
188/188 [==============================] - 1s 2ms/step - loss: 0.3157 - accuracy: 0.8888
processing fold # 3
188/188 [==============================] - 0s 2ms/step - loss: 0.3156 - accuracy: 0.8897
processing fold # 4
188/188 [==============================] - 1s 2ms/step - loss: 0.3041 - accuracy: 0.8945
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.3259 - accuracy: 0.8870
processing fold # 6
188/188 [==============================] - 0s 2ms/step - loss: 0.3167 - accuracy: 0.8972
processing fold # 7
188/188 [==============================] - 0s 2ms/step - loss: 0.3355 - accuracy: 0.8867
processing fold # 8
188/188 [==============================] - 0s 2ms/step - loss: 0.3112 - accuracy: 0.8933
processing fold # 9

In [21]:
all_scores

[0.890666663646698,
 0.8913333415985107,
 0.8888333439826965,
 0.8896666765213013,
 0.8945000171661377,
 0.8870000243186951,
 0.8971666693687439,
 0.8866666555404663,
 0.8933333158493042,
 0.890333354473114]

In [22]:
np.mean(all_scores)

0.8909500062465667

In [7]:
k = 10
num_val = len(X_train) // k
num_epochs = 45
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)





processing fold # 0
188/188 [==============================] - 0s 1ms/step - loss: 0.3516 - accuracy: 0.8787
processing fold # 1
188/188 [==============================] - 0s 1ms/step - loss: 0.3287 - accuracy: 0.8903
processing fold # 2
188/188 [==============================] - 0s 1ms/step - loss: 0.3544 - accuracy: 0.8808
processing fold # 3
188/188 [==============================] - 0s 1ms/step - loss: 0.3257 - accuracy: 0.8895
processing fold # 4
188/188 [==============================] - 0s 1ms/step - loss: 0.3490 - accuracy: 0.8845
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.3555 - accuracy: 0.8893
processing fold # 6
188/188 [==============================] - 0s 1ms/step - loss: 0.3313 - accuracy: 0.8872
processing fold # 7
188/188 [==============================] - 0s 1ms/step - loss: 0.3370 - accuracy: 0.8887
processing fold # 8
188/188 [==============================] - 0s 1ms/step - loss: 0.3463 - accuracy: 0.8865
processing fold # 9

In [8]:
all_scores

[0.8786666393280029,
 0.890333354473114,
 0.8808333277702332,
 0.8895000219345093,
 0.8845000267028809,
 0.8893333077430725,
 0.8871666789054871,
 0.8886666893959045,
 0.8865000009536743,
 0.890333354473114]

In [9]:
np.mean(all_scores)

0.8865833401679992

In [10]:
k = 10
num_val = len(X_train) // k
num_epochs = 50
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 1ms/step - loss: 0.3292 - accuracy: 0.8870
processing fold # 1
188/188 [==============================] - 0s 1ms/step - loss: 0.3080 - accuracy: 0.8945
processing fold # 2
188/188 [==============================] - 0s 1ms/step - loss: 0.3321 - accuracy: 0.8878
processing fold # 3
188/188 [==============================] - 0s 1ms/step - loss: 0.3170 - accuracy: 0.8912
processing fold # 4
188/188 [==============================] - 0s 1ms/step - loss: 0.3288 - accuracy: 0.8870
processing fold # 5
188/188 [==============================] - 0s 1ms/step - loss: 0.3405 - accuracy: 0.8853
processing fold # 6
188/188 [==============================] - 0s 1ms/step - loss: 0.3224 - accuracy: 0.8975
processing fold # 7
188/188 [==============================] - 0s 1ms/step - loss: 0.3275 - accuracy: 0.8927
processing fold # 8
188/188 [==============================] - 0s 1ms/step - loss: 0.3268 - accuracy: 0.8905
processing fold # 9

In [11]:
all_scores

[0.8870000243186951,
 0.8945000171661377,
 0.8878333568572998,
 0.8911666870117188,
 0.8870000243186951,
 0.8853333592414856,
 0.8974999785423279,
 0.8926666378974915,
 0.890500009059906,
 0.8881666660308838]

In [12]:
np.mean(all_scores)

0.8901666760444641

In [13]:
k = 10
num_val = len(X_train) // k
num_epochs = 55
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 2ms/step - loss: 0.3218 - accuracy: 0.8930
processing fold # 1
188/188 [==============================] - 0s 1ms/step - loss: 0.3283 - accuracy: 0.8868
processing fold # 2
188/188 [==============================] - 0s 1ms/step - loss: 0.3762 - accuracy: 0.8777
processing fold # 3
188/188 [==============================] - 0s 1ms/step - loss: 0.3372 - accuracy: 0.8913
processing fold # 4
188/188 [==============================] - 1s 2ms/step - loss: 0.3143 - accuracy: 0.8957
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.3195 - accuracy: 0.8995
processing fold # 6
188/188 [==============================] - 0s 2ms/step - loss: 0.3329 - accuracy: 0.8977
processing fold # 7
188/188 [==============================] - 0s 2ms/step - loss: 0.3516 - accuracy: 0.8900
processing fold # 8
188/188 [==============================] - 0s 1ms/step - loss: 0.4264 - accuracy: 0.8622
processing fold # 9

In [14]:
all_scores

[0.8930000066757202,
 0.8868333101272583,
 0.8776666522026062,
 0.8913333415985107,
 0.8956666588783264,
 0.8995000123977661,
 0.8976666927337646,
 0.8899999856948853,
 0.8621666431427002,
 0.8636666536331177]

In [15]:
np.mean(all_scores)

0.8857499957084656

In [16]:
k = 10
num_val = len(X_train) // k
num_epochs = 75
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 1ms/step - loss: 0.3787 - accuracy: 0.8910
processing fold # 1
188/188 [==============================] - 0s 1ms/step - loss: 0.3261 - accuracy: 0.8995
processing fold # 2
188/188 [==============================] - 0s 1ms/step - loss: 0.3713 - accuracy: 0.8950
processing fold # 3
188/188 [==============================] - 0s 1ms/step - loss: 0.3727 - accuracy: 0.8918
processing fold # 4
188/188 [==============================] - 0s 1ms/step - loss: 0.3648 - accuracy: 0.8962
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.3825 - accuracy: 0.8897
processing fold # 6
188/188 [==============================] - 0s 2ms/step - loss: 0.3631 - accuracy: 0.8968
processing fold # 7
188/188 [==============================] - 0s 2ms/step - loss: 0.3685 - accuracy: 0.8942
processing fold # 8
188/188 [==============================] - 0s 2ms/step - loss: 0.4106 - accuracy: 0.8848
processing fold # 9

In [17]:
all_scores

[0.890999972820282,
 0.8995000123977661,
 0.8949999809265137,
 0.8918333053588867,
 0.8961666822433472,
 0.8896666765213013,
 0.8968333601951599,
 0.8941666483879089,
 0.8848333358764648,
 0.8813333511352539]

In [18]:
np.mean(all_scores)

0.8920333325862885

In [19]:
k = 10
num_val = len(X_train) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 2ms/step - loss: 0.4241 - accuracy: 0.8905
processing fold # 1
188/188 [==============================] - 0s 2ms/step - loss: 0.4009 - accuracy: 0.8920
processing fold # 2
188/188 [==============================] - 0s 2ms/step - loss: 0.4051 - accuracy: 0.8967
processing fold # 3
188/188 [==============================] - 0s 2ms/step - loss: 0.4644 - accuracy: 0.8858
processing fold # 4
188/188 [==============================] - 0s 2ms/step - loss: 0.4391 - accuracy: 0.8887
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.3990 - accuracy: 0.9010
processing fold # 6
188/188 [==============================] - 0s 2ms/step - loss: 0.4117 - accuracy: 0.9022
processing fold # 7
188/188 [==============================] - 0s 2ms/step - loss: 0.4002 - accuracy: 0.9023
processing fold # 8
188/188 [==============================] - 0s 2ms/step - loss: 0.4964 - accuracy: 0.8828
processing fold # 9

In [20]:
all_scores

[0.890500009059906,
 0.8920000195503235,
 0.8966666460037231,
 0.8858333230018616,
 0.8886666893959045,
 0.9010000228881836,
 0.9021666646003723,
 0.9023333191871643,
 0.8828333616256714,
 0.8911666870117188]

In [21]:
np.mean(all_scores)

0.893316674232483

In [22]:
k = 10
num_val = len(X_train) // k
num_epochs = 150
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 2ms/step - loss: 0.6034 - accuracy: 0.8843
processing fold # 1
188/188 [==============================] - 0s 2ms/step - loss: 0.5410 - accuracy: 0.8992
processing fold # 2
188/188 [==============================] - 0s 2ms/step - loss: 0.6295 - accuracy: 0.8812
processing fold # 3
188/188 [==============================] - 1s 2ms/step - loss: 0.5696 - accuracy: 0.8980
processing fold # 4
188/188 [==============================] - 0s 2ms/step - loss: 0.5846 - accuracy: 0.8910
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.5719 - accuracy: 0.9017
processing fold # 6
188/188 [==============================] - 1s 2ms/step - loss: 0.5268 - accuracy: 0.9028
processing fold # 7
188/188 [==============================] - 0s 2ms/step - loss: 0.5804 - accuracy: 0.8945
processing fold # 8
188/188 [==============================] - 0s 2ms/step - loss: 0.6458 - accuracy: 0.8875
processing fold # 9

In [23]:
all_scores

[0.8843333125114441,
 0.8991666436195374,
 0.8811666369438171,
 0.8980000019073486,
 0.890999972820282,
 0.9016666412353516,
 0.9028333425521851,
 0.8945000171661377,
 0.887499988079071,
 0.890999972820282]

In [24]:
np.mean(all_scores)

0.8931166529655457

In [25]:
k = 10
num_val = len(X_train) // k
num_epochs = 300
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 2ms/step - loss: 0.7914 - accuracy: 0.8945
processing fold # 1
188/188 [==============================] - 0s 2ms/step - loss: 0.7240 - accuracy: 0.9022
processing fold # 2
188/188 [==============================] - 0s 2ms/step - loss: 0.8082 - accuracy: 0.8957
processing fold # 3
188/188 [==============================] - 0s 2ms/step - loss: 0.7843 - accuracy: 0.8977
processing fold # 4
188/188 [==============================] - 0s 2ms/step - loss: 0.7788 - accuracy: 0.8998
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.7446 - accuracy: 0.9020
processing fold # 6
188/188 [==============================] - 0s 2ms/step - loss: 0.7513 - accuracy: 0.9040
processing fold # 7
188/188 [==============================] - 0s 2ms/step - loss: 0.8099 - accuracy: 0.9012
processing fold # 8
188/188 [==============================] - 0s 2ms/step - loss: 0.8088 - accuracy: 0.8988
processing fold # 9

In [26]:
all_scores

[0.8945000171661377,
 0.9021666646003723,
 0.8956666588783264,
 0.8976666927337646,
 0.8998333215713501,
 0.9020000100135803,
 0.9039999842643738,
 0.9011666774749756,
 0.8988333344459534,
 0.8968333601951599]

In [27]:
np.mean(all_scores)

0.8992666721343994

In [28]:
k = 10
num_val = len(X_train) // k
num_epochs = 37
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
188/188 [==============================] - 0s 2ms/step - loss: 0.2945 - accuracy: 0.8992
processing fold # 1
188/188 [==============================] - 0s 2ms/step - loss: 0.2995 - accuracy: 0.8933
processing fold # 2
188/188 [==============================] - 0s 2ms/step - loss: 0.3321 - accuracy: 0.8803
processing fold # 3
188/188 [==============================] - 0s 2ms/step - loss: 0.3123 - accuracy: 0.8890
processing fold # 4
188/188 [==============================] - 0s 2ms/step - loss: 0.2946 - accuracy: 0.8937
processing fold # 5
188/188 [==============================] - 0s 2ms/step - loss: 0.3071 - accuracy: 0.8957
processing fold # 6
188/188 [==============================] - 0s 2ms/step - loss: 0.3240 - accuracy: 0.8933
processing fold # 7
188/188 [==============================] - 0s 2ms/step - loss: 0.3201 - accuracy: 0.8912
processing fold # 8
188/188 [==============================] - 0s 2ms/step - loss: 0.3338 - accuracy: 0.8890
processing fold # 9

In [29]:
all_scores

[0.8991666436195374,
 0.8933333158493042,
 0.8803333044052124,
 0.8889999985694885,
 0.893666684627533,
 0.8956666588783264,
 0.8933333158493042,
 0.8911666870117188,
 0.8889999985694885,
 0.8861666917800903]

In [30]:
np.mean(all_scores)

0.8910833299160004

In [ ]:
k = 10
num_val = len(X_train) // k
num_epochs = 350
all_scores = []

for i in range(k):
    print('processing fold #', i)
    
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, verbose=0, )
    
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0


In [ ]:
np.mean(all_scores)